In [1]:
%load_ext rpy2.ipython
import warnings
warnings.filterwarnings('ignore')

### Prep: EMT score calculation
- Input files:
    - `./data/EMT_signature_genes_in_gencode.v31lift37.txt`: file contains 207 EMT signature genes
    - `./CCLE/exp/exp_FPKM_EMT_signature_genes_in_gencode.v31lift37.txt`: file contains the expression value of 207 signature genes for all samples. Dimention genes x samples
- Output matrix:
    - output matrix would be a matrix with two columns (STATISTIC, pval)
        - where STATISTIC represent the KS Statistics, -1 -- more epithelial; 1 -- more mysenchymal
    - the matrix is also stored in `./CCLE/plot_heatmap/EMT_score_CCLE.txt`.

In [41]:
%%R
library(zeallot)
KS_test <- function(x, y) {
    n.x <- length(x)
    n.y <- length(y)

    w <- c(x, y)

    z <- cumsum(ifelse(order(w) <= n.x, 1/n.x, -1/n.y))
    z <- z[c(which(diff(sort(w)) != 0), n.x + n.y)] #exclude ties

    alternative <- "two.sided"
    STATISTIC <- switch(alternative, two.sided = z[which.max(abs(z))], 
                        greater = max(z), less = -min(z))

    test = ks.test(x, y)
    return(c(STATISTIC, test$p.value))
}

exp2EMT <- function(exp, signature) {
    # exp is a matrix of expression value; colname is sample name
    # signature is a vector of EMT signature corresponding to the rows of exp matrix
    result = data.frame(STATISTIC = NULL, pval = NULL)
    for (sample in colnames(exp)) {
        exp_E <- exp[which(signature == 'Epi'), sample]
        exp_M <- exp[which(signature == 'Mes'), sample]
        c(STATISTIC, pval) %<-% KS_test(exp_E, exp_M)
        result = rbind(result, data.frame(STATISTIC= STATISTIC, pval = pval))
    }
    rownames(result) = colnames(exp)
    return(result)
}

In [44]:
%%R
exp <- read.table('./CCLE/exp/exp_FPKM_EMT_signature_genes_in_gencode.v31lift37.txt', header = T, sep = '\t', quote = '', stringsAsFactors = FALSE, row.names = 1)
EMT <- read.table('./data/EMT_signature_genes_in_gencode.v31lift37.txt', header = T, sep = '\t', quote = '', stringsAsFactors = FALSE, row.names = 7)
result = exp2EMT(exp[, -c(1)], EMT[rownames(exp), 'EM_Signature'])

### Plot Heatmap
- Input files
    - EMT score matrix generated by running the code described earlier, or directly from file `./CCLE/plot_heatmap/EMT_score_CCLE.txt`
    - `./CCLE/plot_heatmap/EMT_EMBO.txt`: EMT score retreived from reference paper (EMBO)
    - `./CCLE/plot_heatmap/EMT_NBT.txt`: EMT classification retreived from reference paper (NBT)
    - `./CCLE/plot_heatmap/filtered_SE.MATS.JC_PSI.txt`: PSI value extracted from rMATS-turbo output

#### calculate correlation between PSI value and EMT score
- Output:
    - `./CCLE/plot_heatmap/CCLE_correlation_EMT-filteredPSI.txt`: correlation between EMT score and PSI value
    - There is already one copy in the output folder that users can use directly

In [ ]:
%%R
EMT <- read.table('./CCLE/plot_heatmap/EMT_score_CCLE.txt', header = T, sep = '\t', stringsAsFactor = F, quote = "", check.name = F)
PSI <- read.table('./CCLE/plot_heatmap/filtered_SE.MATS.JC_PSI.txt', header = T, sep = '\t', row.names = 1, quote = "", stringsAsFactor = F)

result = list()
for (ID in rownames(PSI)){
    mydf = data.frame(EMT = EMT$STATISTIC, PSI = as.numeric(PSI[ID, rownames(EMT)]))
    fit <- lm(EMT ~ PSI, data = mydf)
    fit_summary = summary(fit)
    result[[ID]] = setNames(c(fit_summary$r.squared, fit$coefficient[2], fit$coefficient[1]), 
                            c('r.squared', 'coefficient', 'intercept'))
}
myresult = do.call(rbind, result)

# write.table(myresult, './CCLE/plot_heatmap/CCLE_correlation_EMT-filteredPSI.txt', sep = '\t', quote = FALSE, row.names = T, col.names = T)


#### Filter out events whose PSI value is highly correlated with EMT score (R2 > 0.4)
- Output
    - `r2_0.4_SE.MATS.JC_PSI.txt`: PSI values of events highly correlated with EMT scores

In [52]:
%%bash
cd ./CCLE/plot_heatmap
awk -F '\t' '($2 > 0.4) {print $0}' CCLE_correlation_EMT-filteredPSI.txt > r2_0.4_CCLE_correlation_EMT-filteredPSI.txt
sed -i '1s/^/ID\t/' r2_0.4_CCLE_correlation_EMT-filteredPSI.txt
awk -F '\t' 'FNR==NR{a[$1]=$1;next} ($1 in a) {print $0}' r2_0.4_CCLE_correlation_EMT-filteredPSI.txt filtered_SE.MATS.JC_PSI.txt > r2_0.4_SE.MATS.JC_PSI.txt


#### Plot

In [55]:
%%R
library(circlize)
library(ComplexHeatmap)
set.seed(1993)

# EMT score and cell line annotation metadata
EMT_score <- read.table('./CCLE/plot_heatmap/EMT_score_CCLE.txt', header = T, sep = '\t', stringsAsFactor = F, quote = "", check.name = F)
metadata <- read.table('./CCLE/metadata/metadata.txt', header = T, sep = '\t', stringsAsFactor = FALSE, quote = "", check.name = F, row.names = 1)
EMT <- cbind(EMT_score, metadata[rownames(EMT_score), ])
EMT <- EMT[with(EMT, order(Tissue, STATISTIC)),]

# from EMBO paper
EMT_score_EMBO <- read.table('./CCLE/plot_heatmap/EMT_EMBO.txt', header = T, sep = '\t', stringsAsFactor = F, quote = "", check.name = F, row.names = 1)
EMT_score_EMBO = EMT_score_EMBO[rownames(EMT), ]
EMT_EMBO = apply(EMT_score_EMBO, 1, 
      FUN = function(x){
          score = as.numeric(x['Generic.EMT.CL.Ksscore'])
          pval = as.numeric(x['Generic.EMT.CL.pv'])
          if (is.na(score)) {
              EM = NA
          } else if (score < 0) {
              EM = ifelse(pval < 0.05, 'Epi', 'intermediate Epi')
          } else {
              EM = ifelse(pval < 0.05, 'Mes', 'intermediate Mes')
          }
          return(EM)
      })


# from NBT paper
EMT_NBT <- read.table('./CCLE/plot_heatmap/EMT_NBT.txt', header = T, sep = '\t', stringsAsFactor = F, quote = "", check.name = F, row.names = 1)
EMT_NBT <- EMT_NBT[rownames(EMT), ]
EMT_NBT[which(EMT_NBT$EMT_status == 'Epithelial'), 'EMT_status'] = 'Epi'
EMT_NBT[which(EMT_NBT$EMT_status == 'Mesenchymal'), 'EMT_status'] = 'Mes'
EMT_NBT[which(EMT_NBT$EMT_status == 'OtherEMT'), 'EMT_status'] = 'Other'

################ ht_CCLE ################
mydf <- read.table('./CCLE/plot_heatmap/r2_0.4_SE.MATS.JC_PSI.txt', header = T, sep = '\t', row.names = 1, quote = "", stringsAsFactor = F)
mydf_CCLE <- mydf[, rownames(EMT)]
mat <- t(apply(mydf_CCLE, 1, scale))


column_ha = HeatmapAnnotation(#'EMT score' = anno1,
                              'EMT score' = anno_points(EMT$STATISTIC, 
                                                        ylim = c(-1, 1), 
                                                        size = unit(0.8, "mm"),
                                                        gp = gpar(col = ifelse(EMT$STATISTIC > 0 & EMT$pval < 0.05, "dodgerblue4", ifelse(EMT$STATISTIC < -0 & EMT$pval < 0.05, "orangered4", "grey"))),
                                                        axis_param = list(side = "left", at = c(-1, 0, 1), labels = c("Epi -1", "0", "Mes  1"))
                                                         ), 
                              'Tan et al., 2014' = factor(EMT_EMBO, levels = c('Epi', 'intermediate Epi', 'intermediate Mes', 'Mes', 'NA')), 
                              'Klijn et al., 2015' = factor(EMT_NBT$EMT_status, levels = c('Epi', 'Other', 'Mes', 'NA')),
                              'disease stage' = factor(EMT$disease_stage, levels = c('benign_neoplasia', 'primary', 'metastasis')),
                              'tissue' = EMT$Tissue,
                              col = list('disease stage' = c("benign_neoplasia" = "darkgrey", "primary" = "turquoise", "metastasis" = "deeppink"),
                                         'Tan et al., 2014' = c('Epi' = 'orangered4', 'intermediate Epi' = 'orangered', 'intermediate Mes' = 'dodgerblue', 'Mes' = 'dodgerblue4', 'NA' = 'white'), 
                                         'Klijn et al., 2015' = c('Epi' = 'orangered4', 'Other' = 'grey', 'Mes' = 'dodgerblue4', 'NA' = 'white')), 
                              na_col = "white")
lgd_EMT = Legend(labels = c('Epi', 'intermediate', 'Mes'), legend_gp = gpar(fill = c('orangered4', 'grey', 'dodgerblue4')), title = "EMT score")


ht_CCLE  = Heatmap(mat, 
            show_row_names = FALSE,
            na_col = "gray30",
            column_split = EMT$Tissue,
            top_annotation = column_ha,
            # right_annotation = row_ha,
            cluster_columns = F, 
            heatmap_legend_param = list(
                title = "CCLE PSI Z-Score",
                legend_height = unit(4, "cm"),
                #direction = "horizontal",
                title_position = "leftcenter-rot"),
            column_title = 'Cancer Cell Line Encyclopedia (CCLE)'
        )

################ PLOT ################
pdf("./CCLE/plot_heatmap/plot_heatmap_sortedByEMT_usingCorrelation.pdf", width = 15, height = 6)
lgd_list = list('EMT score' = lgd_EMT)

ht = draw(
    ht_CCLE,
    annotation_legend_list = lgd_list,
    annotation_legend_side = "right",
    heatmap_legend_side = "right"
)

dev.off()

png 
  2 
